In [ ]:
import os

import loguru
import polars as pl
from polars.exceptions import ComputeError
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError, OperationalError, DataError
from dotenv import load_dotenv

load_dotenv()
logger = loguru.logger

In [ ]:
__thick_mode = os.getenv("THICK_MODE")
__connect_args = {
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "service_name": os.getenv("DB_SERVICE_NAME"),
}

In [ ]:
class OracleConnector:
    def __init__(self, thick_mode: bool, connection_args: dict):
        self.__thick_mode = thick_mode
        self.__connect_args = connection_args

        self.__engine = create_engine(
            "oracle+oracledb://:@",
            thick_mode=self.__thick_mode,
            connect_args=self.__connect_args
        )
        
    def execute(self, query: str) -> pl.DataFrame | None:
        result = None
        
        try:
            with self.__engine.connect() as connection:
                result = pl.read_database(query=text(query), connection=connection)
        except OperationalError as e:
            logger.error(f"데이터베이스 연결 오류: {e}")
        except DataError as e:
            logger.error(f"데이터 형식 오류: {e}")
        except ComputeError as e:
            logger.error(f"Polars 계산 오류: {e}")
        except SQLAlchemyError as e:
            logger.error(f"SQLAlchemy 오류: {e}")
        except Exception as e:
            logger.error(f"Error while executing query: {e}")
        
        return result

In [ ]:
connector = OracleConnector(__thick_mode, __connect_args)

In [ ]:
connector.execute("SELECT * FROM MY_TABLE")